In [135]:
import numpy as np
import struct as struct
from struct import unpack

In [136]:
filename = 'random/4x4x4x4_random'
conf = []
count = 0
with open(filename, "rb") as f:
    while True:
        if count == 0:
            struct_fmt = '=4id' 
            struct_len = struct.calcsize(struct_fmt)
            struct_unpack = struct.Struct(struct_fmt).unpack_from #reads lattice dimensions and plaquete 0.0
            count += 1
            data = f.read(struct_len)
            s = struct_unpack(data)
            print('Lattice dimensions Nt={0}, Nz={1}, Ny={2}, Nx={3}'.format(s[0],s[1],s[2],s[3]))
        elif count == 1:
            struct_fmt = '=18d' 
            struct_len = struct.calcsize(struct_fmt)
            struct_unpack = struct.Struct(struct_fmt).unpack_from
            data = f.read(struct_len)
            if not data: break   
            s = struct_unpack(data)
            conf.append(s) #SU(3) elements, i.e. 18 double (real) numbers

Lattice dimensions Nt=4, Nz=4, Ny=4, Nx=4


In [137]:
n = 0
print(type(conf[n]))
N = len(conf[n])
print('No of elements',N)
print('Gauge link number',n,'\n')
for i in range(N):
    print(conf[n][i])

<class 'tuple'>
No of elements 18
Gauge link number 0 

0.9999999746315572
0.00013385364474259603
1.6921136397110694e-05
-8.287776989964238e-05
-0.00011064133983748479
0.00011585996258985477
-1.6924289310571552e-05
-8.288589022153253e-05
0.9999999923955478
-4.539150007683379e-05
-7.509335709422385e-05
1.8788331263353676e-05
0.00011063636640867454
0.00011585844239530272
7.510734575802598e-05
1.8771069508758602e-05
0.999999980258699
-8.846214467548891e-05


$$
\gamma_3 = \gamma_t = \begin{pmatrix} 
0 & 0 & 1 & 0 \\
0 & 0 & 0 & 1 \\
1 & 0 & 0 & 0 \\
0 & 1 & 0 & 0
\end{pmatrix}, \quad \gamma_2 = \gamma_z = \begin{pmatrix} 
0 & 0 & i & 0 \\
0 & 0 & 0 & -i \\
-i & 0 & 0 & 0 \\
0 & i & 0 & 0
\end{pmatrix}, \quad \gamma_1 = \gamma_y = \begin{pmatrix} 
0 & 0 & 0 & -1 \\
0 & 0 & 1 & 0 \\
0 & 1 & 0 & 0 \\
-1 & 0 & 0 & 0
\end{pmatrix}, \quad \gamma_0 = \gamma_x = \begin{pmatrix} 
0 & 0 & 0 & i \\
0 & 0 & i & 0 \\
0 & -i & 0 & 0 \\
-i & 0 & 0 & 0
\end{pmatrix}
$$

In [142]:
#Dirac matrices
gamma0 = np.array([[0,0,0,1j],[0,0,1j,0],[0,-1j,0,0],[-1j,0,0,0]])
gamma1 = np.array([[0,0,0,-1],[0,0,1,0],[0,1,0,0],[-1,0,0,0]])
gamma2 = np.array([[0,0,1j,0],[0,0,0,-1j],[-1j,0,0,0],[0,1j,0,0]])
gamma3 = np.array([[0,0,1,0],[0,0,0,1],[1,0,0,0],[0,1,0,0]])
print(gamma0)
print(gamma1)
print(gamma2)
print(gamma3)
gammaMatrices = [gamma0,gamma1,gamma2,gamma3]

[[ 0.+0.j  0.+0.j  0.+0.j  0.+1.j]
 [ 0.+0.j  0.+0.j  0.+1.j  0.+0.j]
 [ 0.+0.j -0.-1.j  0.+0.j  0.+0.j]
 [-0.-1.j  0.+0.j  0.+0.j  0.+0.j]]
[[ 0  0  0 -1]
 [ 0  0  1  0]
 [ 0  1  0  0]
 [-1  0  0  0]]
[[ 0.+0.j  0.+0.j  0.+1.j  0.+0.j]
 [ 0.+0.j  0.+0.j  0.+0.j -0.-1.j]
 [-0.-1.j  0.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  0.+1.j  0.+0.j  0.+0.j]]
[[0 0 1 0]
 [0 0 0 1]
 [1 0 0 0]
 [0 1 0 0]]


In [149]:
#Kronecker delta. This also works in 4d as long as u = [ux,uy,uz,ut], y = [vx,vy,vz,v]
def kDelta(u,v):
    if u == v:
        return 1
    else:
        return 0

def DiracEntry(alfa,beta,a,b,n,m,m0,h):
    """
    alfa, beta: 0,1,2,3 (spin indices)
    c, d: 0,1,2 (color indices)
    n, m: 0,1,...,Nx Ny Nz Nt (volume)
    m0: bare mass
    h: lattice spacing
    """
    #Conf[coord(n,mu)] gives U_mu(n)
    I = np.identity(4)
    hat = [np.array([1,0,0,0]),np.array([0,1,0,0]),np.array([0,0,1,0]),np.array([0,0,0,1])]
    Dnm = (m0+4)/a * kDelta(alfa,beta)*kDelta(a,b)*kDelta(n,m)
    #sum over mu
    for mu in range(4):
        Dnm += -0.5 * a * (I-gammaMatrices[mu])[alfa,beta] * conf[coord(n,mu)][a,b] * kDelta(n+hat[mu],m) 
        Dnm += -0.5 * a * (I+gammaMatrices[mu])[alfa,beta] * conf[coord(n-hat[mu],mu)][b,a].conj() * kDelta(n-hat[mu],m)
    return Dnm

def DiracMatrix(Nt,Nx,Ny,Nz):
    #D = np.zeros((12*Nt*Nx*Ny*Nz,12*Nt*Nx*Ny*Nz))
    for t in range(Nt):
        for z in range(Nz):
            for y in range(Ny):
                for x in range(Nx):
                    for alfa in range(4):
                        for beta in range(4):
                            for a in range(3):
                                for b in range(3):
                                    DiracEntry(alfa,beta,a,b,n,m,m0,h)

In [165]:
#This allows us to identify the matrix rows or columns with the space-time and color coordinates (x,y,z,t,c,s)
Indices = {'x':[],'y':[],'z':[],'t':[],'c':[],'s':[]}
C, S = 3, 4 #Color (related to the dimension of U_mu) and S (related to the dimension of gamma)
Nt, Nx, Ny, Nz = 4, 4, 4, 4
for t in range(Nt):
    for z in range(Nz):
        for y in range(Ny):
            for x in range(Nx):
                for s in range(S):
                    for c in range(C):
                        #print(t*Nz*Ny*Nx*C*S + z*Ny*Nx*S*C + y*Nx*S*C + x*S*C + s*C + c) #Coordinates vectorization
                        Indices['x'].append(x)
                        Indices['y'].append(y)
                        Indices['z'].append(z)
                        Indices['t'].append(t)
                        Indices['c'].append(c)
                        Indices['s'].append(s)

In [ ]:
#This allows us to identify the matrix rows or columns with the space-time and color coordinates (x,y,z,t,c,s)
Indices = {'x':[],'y':[],'z':[],'t':[],'c':[],'s':[]}
C, S = 3, 4 #Color (related to the dimension of U_mu) and S (related to the dimension of gamma)
for x in range(Ns):
    for y in range(Ns):
        for z in range(Ns):
            for t in range(Nt):
                for c in range(C):
                    for s in range(S):
                        #print(x*Ns*Ns*Nt*C*S + y*Ns*Nt*C*S + z*Nt*C*S + t*C*S + c*S + s) Coordinates vectorization
                        Indices['x'].append(x)
                        Indices['y'].append(y)
                        Indices['z'].append(z)
                        Indices['t'].append(t)
                        Indices['c'].append(c)
                        Indices['s'].append(s)